<html>
    <div style="color:#363945; background-color:#E1F6FF; display: block">
        <h1> Preprocessing Data - Notebook Summary </h1>
            <ol>
                <li> Split answers separated by semicolons in different columns to a list of single answers.</li>
                <li> Replace string values in Numeric columns.</li>
            </ol><br>
    </div>
</html>

In [1]:
# Constants
DATA_PATH = "../data/raw/survey_results_public.csv"
EXPORT_PATH = "../data/processed/01_preprocessed.pkl"

# Replacement Dictionary to replace string values in numeric columns
REPLACEMENT_DICT = {
 'Age1stCode' : {'Younger than 5 years': 4, 'Older than 85' : 86},  
 'YearsCode' :  {'Less than 1 year' : 0, 'More than 50 years' : 51},
 'YearsCodePro': {'Less than 1 year' : 0, 'More than 50 years' : 51}
}

In [2]:
import numpy as np
import pandas as pd
import pickle

## Functions

In [3]:
def split_answers(data_series, delimiter = ';'):
    """
    Function to split answers in a single string into a list of single strings each representing an answer.
    Input:
    data_series: A pandas data series that contains the answers in a single string.
    delimiter: A string that represents the delimiter that separates multiple answers inside a string. 
               Defaults to ";"
    Returns: 
    A modified pandas data series. (If Column contains)
    """
    
    def is_splittable(pd_series, delimiter):
        """ Return a Boolean series to check wether the string contains multiple answers or just a single answer.
            True: Contains multiple answers.
            False: Contains a Single answer.
        """
        return pd_series.str.contains(delimiter)
    
    def split_answer(pd_series, delimiter):
        """Function that splits a single answer"""
        return pd_series.str.split(delimiter)
    
    #-------------------------------------------------
    
    # Check if the answer has multiple answers. If not Returns: Original
    splittable_values = is_splittable(data_series, delimiter)
    if not splittable_values.any(): 
        return data_series   # all series have no multiple answers.
    
    # Else: Split each answer to a list and return modified series
    else:
        modified_series = split_answer(data_series, delimiter)
        
    # Replace NAs with empty lists
    mask_null = modified_series.isnull()
    modified_series.loc[mask_null] = modified_series.loc[mask_null].apply(lambda x: [])
    
    # Return Modified series
    return modified_series

## Preprocessing

In [4]:
# Read data and make a copy of it
raw_data = pd.read_csv(DATA_PATH)
df = raw_data.copy()

### 1. Split answers 

In [5]:
# Get 'object' type column names 
object_cols = df.select_dtypes(include= "object").columns.tolist()
type(object_cols)
for col in object_cols:
    df[col] = split_answers(df[col])

### 2. Replace string values with numbers

In [6]:
for col, replacement in REPLACEMENT_DICT.items():
    df[col] = df[col].replace(replacement).astype(np.float32)

In [7]:
# Showing the output of affected columns 
df[['Age1stCode','YearsCode','YearsCodePro', 'DevType', 
    'LanguageWorkedWith',    'DatabaseWorkedWith',    'WebframeWorkedWith',    'MiscTechWorkedWith',
   'LanguageDesireNextYear','DatabaseDesireNextYear','WebframeDesireNextYear','MiscTechDesireNextYear']].head()

,Age1stCode,YearsCode,YearsCodePro,DevType,LanguageWorkedWith,DatabaseWorkedWith,WebframeWorkedWith,MiscTechWorkedWith,LanguageDesireNextYear,DatabaseDesireNextYear,WebframeDesireNextYear,MiscTechDesireNextYear
0,13.0,36.0,27.0,"[Developer, desktop or enterprise applications...","[C#, HTML/CSS, JavaScript]","[Elasticsearch, Microsoft SQL Server, Oracle]","[ASP.NET, ASP.NET Core]","[.NET, .NET Core]","[C#, HTML/CSS, JavaScript]",[Microsoft SQL Server],[ASP.NET Core],"[.NET Core, Xamarin]"
1,19.0,7.0,4.0,"[Developer, full-stack, Developer, mobile]","[JavaScript, Swift]",[],[],[React Native],"[Python, Swift]",[],[],"[React Native, TensorFlow, Unity 3D]"
2,15.0,4.0,NaN,[],"[Objective-C, Python, Swift]",[],[],[],"[Objective-C, Python, Swift]",[],[],[]
3,18.0,7.0,4.0,[],[],[],[],[],[],[],[],[]
4,16.0,15.0,8.0,[],"[HTML/CSS, Ruby, SQL]","[MySQL, PostgreSQL, Redis, SQLite]",[Ruby on Rails],[Ansible],"[Java, Ruby, Scala]","[MySQL, PostgreSQL]","[Django, Ruby on Rails]","[Ansible, Chef]"


## Export to pickle file

In [8]:
df.to_pickle(EXPORT_PATH)